### Packages

In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import numpy.random as nr
import itertools as it
import operator as op
import random as rd

### Input

In [2]:
N = int(input('Input the number of airports'))
NOF = int(input('Input the number of flights'))

Input the number of airports 25
Input the number of flights 5


### Data

In [3]:
toc = dt.datetime.now() #program start time

nr.seed(20220606)
rd.seed(20220606)

#flight costs matrix
C = 708 + (51124 - 708)*nr.rand(N**2).reshape(N,N)
C -= np.diag(np.diag(C)) 
        
interval = {}
interval["Acheck"] = nr.randint(56, 71)
interval["Bcheck"] = nr.randint(180, 241)
interval["Ccheck"] = nr.randint(540, 721)
interval["Dcheck"] = nr.randint(2160, 3601)
        
durations = {}
durations["Acheck"] = [1]*N
durations["Bcheck"] = nr.randint(1, 4, size = N)
durations["Ccheck"] = nr.randint(7, 15, size = N)
durations["Dcheck"] = nr.randint(21, 43, size = N)
        
costs = {}
costs["Acheck"] = (310 + (820 - 310)*nr.rand(N))
costs["Bcheck"] = (4960 + (7380 - 4960)*nr.rand(N))
costs["Ccheck"] = (186000 + (246000 - 186000)*nr.rand(N))
costs["Dcheck"] = (930000 + (2050000 - 930000)*nr.rand(N))

MFPD = 4 #maximum flights per day
home = nr.randint(N) #initial airport
I = set(np.arange(N)) #airport set
MFCPD = C.max() #maximum flight costs per day
MFC = (C + np.diag([np.inf]*N)).min() #minimum flight costs
MDP = (MFCPD + MFPD*MFC)/2 #median daily profit

### Flight Simulation

In [4]:
def maintenance_check(day, maintenance, airport):
    for check in maintenance["amount"].keys():
        if day >= (maintenance["amount"][check]+1)*interval[check] :
            
            maintenance["amount"][check] += 1
            day += durations[check][airport]
            
            maintenance["cost"][check] += costs[check][airport]
            maintenance["loss"][check] += durations[check][airport]*MDP
            
    return day, maintenance

In [5]:
def total_cost(route):
    day = 0 # aircraft age
        
    maintenance = {}
    maintenance["loss"]     = {"Acheck" : 0, "Bcheck" : 0, "Ccheck" : 0, "Dcheck" : 0}
    maintenance["amount"]   = {"Acheck" : 0, "Bcheck" : 0, "Ccheck" : 0, "Dcheck" : 0}
    maintenance["cost"]     = {"Acheck" : 0, "Bcheck" : 0, "Ccheck" : 0, "Dcheck" : 0}
        
    FC = 0 #flight cost
    
    FPD = 0 #flights per day
    FCPD = 0 #flight costs per day
        
    for i in range(1,len(route)): 
        FCPD += C[route[i-1]][route[i]]
        FPD += 1
        
        if (FPD > MFPD) or (FCPD > MFCPD):
            FC += (FCPD - C[route[i-1]][route[i]])
            day += 1
            
            FPD = 1
            FCPD = C[route[i-1]][route[i]]
            day, maintenance = maintenance_check(day, maintenance, route[i-1])
            
    FC += FCPD
    if route[-1] == home:
        day += 1
        day, maintenance = maintenance_check(day, maintenance, route[-1])
    
    objective = FC + sum(list(maintenance["cost"].values()))
    objective += sum(list(maintenance["loss"].values()))

    return day, FC, maintenance, objective

### Held Karp

In [6]:
def held_karp(A, TotalFlights, route, last):

    objective = np.inf
    B = dict() #dummy memo containing objectives and routes
    
    #for TotalFlights < 3 the algorithm is the same as brute force 
    #because the elements of the set only have 1 member (home,{i},j)
    if TotalFlights < 3:
        for S in it.permutations(I - set(route), TotalFlights):
                        
            dummy = total_cost(route + S + last)
            dummyO = dummy[-1] 
            B[S] = (dummyO, S) 
            
            if dummyO < objective:
                PartialRoute = S + last
                day, FC, maintenance, objective = dummy
            
    else:
        for S in it.combinations(I - set(route), TotalFlights-1):
            for i in (I - set(route) - set(S)):
                
                B[tuple(sorted(S)) + (i,)] = (np.inf, S + (i,))
                    
                for j in it.combinations(S, TotalFlights-2):
                    DummyRoute = A[tuple(sorted(j)) + tuple(set(S) - set(j))][1]  + (i,)
                    
                    dummy2 = total_cost(route + DummyRoute + last)
                    dummy2O = dummy2[-1]
                        
                    if  dummy2O < B[tuple(sorted(S)) + (i,)][0] :
                        B[tuple(sorted(S)) + (i,)] =  (dummy2O, DummyRoute) 
                
                dummy = total_cost(route + B[tuple(sorted(S)) + (i,)][1] + last)
                dummyO = dummy[-1] 
                        
                if dummyO < objective :
                    PartialRoute = B[tuple(sorted(S)) + (i,)][1] + last
                    day, FC, maintenance, objective = dummy
    
    return B, PartialRoute

### Main Program

In [7]:
route = (home,)

TotalFlights = 2
A = {}

while set(route) != I:
    
    if TotalFlights >= N - len(route):
        B, PartialRoute = held_karp(A, N - len(route), route, (home,))    
        route = route + PartialRoute
    
    else :
        B, PartialRoute = held_karp(A, TotalFlights, route, ()) 
    
        if TotalFlights == NOF:
            route = route + PartialRoute
            TotalFlights = 2
            
        else:
            TotalFlights += 1 

    A = B  #memo containing objective and route

day, FC, maintenance, objective = total_cost(route)

In [8]:
def output():
    print('Total cost of flights  $%d' 
        %(FC))
    print('Total maintenance costs $%d' 
        %(sum(list(maintenance["cost"].values()))))
    print('Total maintenance loss $%d' 
        %(sum(list(maintenance["loss"].values()))))

    print('Total cost $%d' 
        %(FC + sum(list(maintenance["cost"].values()))))
    print()
    print("\033[1m" +"Details :" +"\033[0m")
    print(f'flight duration {int(day)} day')
    print()
    print('Number of The A check maintenance = %d' 
        %(maintenance["amount"]['Acheck']))
    print('Number of The B check maintenance = %d' 
        %(maintenance["amount"]['Bcheck']))
    print('Number of The C check maintenance = %d' 
        %(maintenance["amount"]['Ccheck']))
    print('Number of The D check maintenance = %d' 
        %(maintenance["amount"]['Dcheck']))
    print()
    print('Costs of The A check maintenance = $%d' 
        %(maintenance["cost"]['Acheck']))
    print('Costs of The B check maintenance = $%d' 
        %(maintenance["cost"]['Bcheck']))
    print('Costs of The C check maintenance = $%d' 
        %(maintenance["cost"]['Ccheck']))
    print('Costs of The D check maintenance = $%d' 
        %(maintenance["cost"]['Dcheck']))
    print()
    print('Loss of The A check maintenance = $%d' 
        %(maintenance["loss"]['Acheck']))
    print('Loss of The B check maintenance = $%d' 
        %(maintenance["loss"]['Bcheck']))
    print('Loss of The C check maintenance = $%d' 
        %(maintenance["loss"]['Ccheck']))
    print('Loss of The D check maintenance = $%d' 
        %(maintenance["loss"]['Dcheck']))
    print()

    print("route : ")
    for airport in route[:-1]:
        print(f'{airport} -> ', end ='')
       
    print(home)
    print()
    print("Execution time : ", dt.datetime.now()-toc)

def detail():
    print('Detail: ')
    FC = 0
    FC_per_day = 0
    flight_per_day = 0 
    day = 0
    for i in range(1,len(route)):
        FC_per_day += C[route[i-1]][route[i]]
        flight_per_day += 1
    
        if (flight_per_day > MFPD) or (FC_per_day > MFCPD):
            FC += (FC_per_day - C[route[i-1]][route[i]])
            day += 1
        
            print('num: '+str(day)+', airport visited: '+str(route[i-flight_per_day :i])+', flight costs: $'+str(FC_per_day - C[route[i-1]][route[i]]))
            FC_per_day = C[route[i-1]][route[i]]
            flight_per_day = 1
        
    FC += FC_per_day
    day += 1
        
    print('num: '+str(day)+', airport visited: '+str(route[-flight_per_day-1 :])+', flight costs: $'+str(FC_per_day))
    print()
    print("Total cost of flights: $"+str(int(FC)))

### Output

In [9]:
output()

Total cost of flights  $194466
Total maintenance costs $0
Total maintenance loss $0
Total cost $194466

Details :
flight duration 8 day

Number of The A check maintenance = 0
Number of The B check maintenance = 0
Number of The C check maintenance = 0
Number of The D check maintenance = 0

Costs of The A check maintenance = $0
Costs of The B check maintenance = $0
Costs of The C check maintenance = $0
Costs of The D check maintenance = $0

Loss of The A check maintenance = $0
Loss of The B check maintenance = $0
Loss of The C check maintenance = $0
Loss of The D check maintenance = $0

route : 
2 -> 23 -> 5 -> 20 -> 21 -> 9 -> 17 -> 15 -> 4 -> 22 -> 16 -> 3 -> 1 -> 14 -> 10 -> 18 -> 13 -> 0 -> 8 -> 19 -> 24 -> 11 -> 12 -> 7 -> 6 -> 2

Execution time :  0:00:32.075336


### Detail

In [10]:
detail()

Detail: 
num: 1, airport visited: (2, 23, 5, 20, 21), flight costs: $7021.081082335734
num: 2, airport visited: (21, 9, 17, 15, 4), flight costs: $10540.721060661119
num: 3, airport visited: (4, 22, 16, 3, 1), flight costs: $14439.182339692228
num: 4, airport visited: (1, 14, 10, 18, 13), flight costs: $23004.284253409394
num: 5, airport visited: (13, 0, 8, 19, 24), flight costs: $26782.110140655066
num: 6, airport visited: (24, 11, 12), flight costs: $41121.40195419865
num: 7, airport visited: (12, 7, 6), flight costs: $40310.510853231805
num: 8, airport visited: (6, 2), flight costs: $31247.399297391465

Total cost of flights: $194466
